# How to transform shapes in 2D...
## ... with the little help of linear algebra!

In [ ]:
from numpy import array, identity as I
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from math import pi as π, cos, sin
from random import random

# A bit of randomness...
σ = lambda x = .5: x*(random() - 0.5)

# Transformation of the polygon's vertices
transform = lambda P, points: array([P @ p for p in points])
# Homogeneous to Euclidean coordinates conversion
h2e = lambda X: array([(x/x[-1])[:-1] for x in X])

### A square (a kinky one...) 

Various $3\times 3$ matrices
$$
\mathbf{I}=
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$$
transform vectors $\left[x, y, 1\right]^T$ of 2D point in homogenous coordinates (this matrix does nothing).

In [ ]:
# The original shape at the origin
sqwk = array([[0, 0, 1], [1, 0, 1], [1, 1, 1], [.25, .25, 1], [0, 1, 1]])

# Identity transformation (a.k.a. "do nothing but with great style!")
sqwk = transform(I(3), sqwk)
verticesI = h2e(sqwk)

## Translation/shift

Thanks to the homogeneous coordinates, translation by $\left({\color{red}{K}}, {\color{blue}{L}}\right)$ is a linear operation!
$$
\mathbf{T}_{K, L}=
\begin{bmatrix}
1 & 0 & {\color{red}{K}}\\
0 & 1 & {\color{blue}{L}}\\
0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
# Our square after translation by K and L...
K, L = 1.5 + σ(), 1 + σ(2)
T = I(3); _T = array([K, L])
T[:2, 2] = _T.T # Transposition seems superfluous here, but it's a good habit

## Rotation

Here is a formula of a rotation matrix (by an angle ${\color{red}{\alpha}}$)
$$
\mathbf{R}_{\alpha}=
\begin{bmatrix}
\cos{\color{red}{\alpha}} & -\sin{\color{red}{\alpha}} & 0\\
\sin{\color{red}{\alpha}} & \cos{\color{red}{\alpha}} & 0\\
0 & 0 & 1
\end{bmatrix}
$$
Observe that $\det(R) = \cos^2{\color{red}{\alpha}} + \sin^2{\color{red}{\alpha}} = 1$:

In [ ]:
# ... and after rotation by α degrees...
α = σ(30)*π/180
R = I(3); _R = array([[cos(α), -sin(α)],
                      [sin(α),  cos(α)]])
R[:2, :2] = _R

## Scaling, squeezing, stretching & shearing – [Procrustes](https://en.wikipedia.org/wiki/Procrustes) would've loved it!

These matrices look sssstraighforward:
$$
\mathbf{S}=
\begin{bmatrix}
{\color{red}{s}} & 0 & 0\\
0 & {\color{red}{s}} & 0\\
0 & 0 & 1
\end{bmatrix},\quad
\mathbf{S1}=
\begin{bmatrix}
{\color{red}{k}} & 0 & 0\\
0 & {\color{red}{\kappa}} & 0\\
0 & 0 & 1
\end{bmatrix},\quad
\mathbf{S2}=
\begin{bmatrix}
{\color{red}{\kappa}} & 0           & 0\\
0      & {\color{red}{\kappa^{-1}}} & 0\\
0 & 0 & 1
\end{bmatrix},\quad
\mathbf{S2}=
\begin{bmatrix}
1      & {\color{red}{\kappa}} & 0\\
{\color{red}{\kappa}} & 1      & 0\\
0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
# ... and scaling...
s = σ(2)
S = I(3); _S = array([[s, 0],
                      [0, s]])
S[:2, :2] = _S

## And...
#  See https://en.wikipedia.org/wiki/Transformation_matrix#/media/File:2D_affine_transformation_matrix.svg
#  and https://en.wikipedia.org/wiki/Transformation_matrix#/media/File:Perspective_transformation_matrix_2D.svg

# ... stretching:
k, κ = 2 + σ(), 2 + σ()
S1 = array([[k, 0, 0],
            [0, κ, 0],
            [0, 0, 1]])
# ... squeezing:
S2 = array([[1/κ, 0, 0],
            [0,   κ, 0],
            [0,   0, 1]])
# and shearing:
S3 = array([[1, k, 0],
            [κ, 1, 0],
            [0, 0, 1]])

## Reflections (ThnX, [Hauseholder](https://en.wikipedia.org/wiki/Alston_Scott_Householder)!)

This form is not exactly obvious that this matrix implements reflection about the line from origin $(0, 0)$ through $({\color{red}{l_x}}, {\color{red}{l_y}})$. It will be clear, however, once we derive a projection formula
$$
\mathbf{S}=
\begin{bmatrix}
{\color{red}{l_x^2 - l_y^2}} & {\color{red}{2l_xl_y}} & 0\\
{\color{red}{2l_xl_y}} & {\color{red}{l_x^2 - l_y^2}} & 0\\
0 & 0 & 1
\end{bmatrix}

In [ ]:

# Reflecting:
lx, ly = .5 + σ(), .5 + σ()
M = I(3)
_M = array([[lx**2 - ly**2,     2*lx*ly  ],
            [    2*lx*ly  , ly**2 - lx**2]])
_M = _M/(lx**2 + ly**2)
M[:2, :2] = _M

## All in a single step!

Regardless how many transformation we need to perform, we can compose them and turn into a single matrix
$$
X' = {\color{red}{\Theta}} X,\quad {\color{red}{\Theta}} = MTSRS_3
$$

In [ ]:
# Feel free to further squeeze, stretch, shear the shape (and mix the order too)...
sqwk = transform(M@T@S@R@S3, sqwk)
verticesII = h2e(sqwk)


# Projections
Perhaps surprisingly, given a form of Hausholder's matrix, that perspective projection matrix is fairly simple. 
$$
\mathbf{P}=
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
{\color{red}{A}} & {\color{red}{B}} & 0
\end{bmatrix}
$$
Here we project a homogeneous points onto a line such that ${\color{red}{A}}x + {\color{red}{B}}y = 1$ (with a center at origin $(0, 0)$):

In [ ]:
# Set the stage!
# https://www.geeksforgeeks.org/how-to-draw-shapes-in-matplotlib-with-python/
fig, ax = plt.subplots()
ax.set_xlim(-1, 4); ax.set_ylim(-1, 4); ax.set_aspect('equal'); plt.grid(True)

# Eventually, the final projection (a.k.a. casting a shadow)...
# https://youtu.be/27vT-NWuw0M, https://youtu.be/JK-8XNIoAkI and https://youtu.be/cTyNpXB92bQ
# which is a linear operation too and can be represented by a matrix as well:
# https://wrfranklin.org/pmwiki/Main/HomogeneousCoords
# The shadow of the final shape on the line Ax + By = 1
A, B = 1 + σ(), 1 + σ()
P = array([[1, 0, 0],
           [0, 1, 0],
           [A, B, 0]])
shdw = transform(P, sqwk); verticesIII = h2e(shdw)

# https://stackoverflow.com/questions/44526364/fill-matplotlib-polygon-with-a-gradient-between-vertices
for (vertices, edges, fills) in [(verticesI, 'green', 'lightgreen'),
                                 (verticesII, 'blue', 'lightblue'),
                                 (verticesIII, 'red', 'tomato')]:
    polygon = Polygon(vertices, ec = edges, fc = fills, alpha = 0.5)
    ax.scatter(vertices[:, 0], vertices[:, 1], color = edges, s = 25)
    ax.add_patch(polygon)


# And the ♪♫Moonlight shadows♫♪... https://youtu.be/ixExC-Zgyzc and https://youtu.be/e80qhyovOnA of the vertices.
# Is that song about Évariste Galois https://en.wikipedia.org/wiki/%C3%89variste_Galois#Final_days?
for v in sqwk:
    # Homogeneous to Euclidean coordinates
    x, y = v[:-1]/v[-1]
    ax.plot((0, x), (0, y), c = 'lightgray', lw = .5, ls = 'dashed')
for v in shdw:
    # Homogeneous to Euclidean coordinates
    x, y = v[:-1]/v[-1]
    ax.plot((0, x), (0, y), c = 'gray', lw = .5, ls = 'dotted')

plt.title('Matrix transforms in 2D: rigid, linear, affine & projective'); plt.show()